# Data Cleaning

Main cleaning steps:
- Transfrom the feature `horsepower` to numeric
- Clean company names in the feature can names
- Transform `origin` column encoded as 1,2,3 to USA, EUROPE, ASIA in `region` column and drop the `origin` column. 

In [1]:
# use black formatter
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# reload module in development mode
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

####  Download data

In [3]:
from src.utils import download_data

download_data()

downloading data to ../data/raw/auto-mpg.data


<IPython.core.display.Javascript object>

#### Load raw data 

In [4]:
import pandas as pd
from src.clean import load_raw_data

df = load_raw_data()
df.head(5)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


<IPython.core.display.Javascript object>

#### Data types and missing values 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
mpg             398 non-null float64
cylinders       398 non-null int64
displacement    398 non-null float64
horsepower      398 non-null object
weight          398 non-null float64
acceleration    398 non-null float64
year            398 non-null int64
origin          398 non-null int64
name            398 non-null object
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB


<IPython.core.display.Javascript object>

#### Transform horsepower column to numeric 

In [6]:
df["horsepower"] = pd.to_numeric(df["horsepower"], errors="coerce")
df[df["horsepower"].isnull()]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
32,25.0,4,98.0,NaN,2046.0,19.0,71,1,ford pinto
126,21.0,6,200.0,NaN,2875.0,17.0,74,1,ford maverick
330,40.9,4,85.0,NaN,1835.0,17.3,80,2,renault lecar deluxe
336,23.6,4,140.0,NaN,2905.0,14.3,80,1,ford mustang cobra
354,34.5,4,100.0,NaN,2320.0,15.8,81,2,renault 18i
374,23.0,4,151.0,NaN,3035.0,20.5,82,1,amc concord dl


<IPython.core.display.Javascript object>

Some horsepwer values are null. We will impute the missing values with the mean value. Once we have split the data into training and test sets.

#### Get company names from car name column

In [7]:
df["name"] = df["name"].str.title()
df["company"] = df["name"].str.split(" ").str[0]
# check companies name
df["company"].value_counts().sort_index()

Amc              28
Audi              7
Bmw               2
Buick            17
Cadillac          2
Capri             1
Chevroelt         1
Chevrolet        43
Chevy             3
Chrysler          6
Datsun           23
Dodge            28
Fiat              8
Ford             51
Hi                1
Honda            13
Maxda             2
Mazda            10
Mercedes          1
Mercedes-Benz     2
Mercury          11
Nissan            1
Oldsmobile       10
Opel              4
Peugeot           8
Plymouth         31
Pontiac          16
Renault           5
Saab              4
Subaru            4
Toyota           25
Toyouta           1
Triumph           1
Vokswagen         1
Volkswagen       15
Volvo             6
Vw                6
Name: company, dtype: int64

<IPython.core.display.Javascript object>

#### Correct car companies names 

There are some typos in company names. For example: `Vokswagen` or `Toyouta`. 
The function `correct_company_names` will do this for us.

In [8]:
from src.clean import correct_company_names

correct_company_names(df)
# check the output
df["company"].value_counts().sort_index()

AMC           28
Audi           7
BMW            2
Buick         17
Cadillac       2
Capri          1
Chevrolet     47
Chrysler       6
Datsun        23
Dodge         28
Fiat           8
Ford          51
Hi             1
Honda         13
Mazda         12
Mercedes       3
Mercury       11
Nissan         1
Oldsmobile    10
Opel           4
Peugeot        8
Plymouth      31
Pontiac       16
Renault        5
Saab           4
Subaru         4
Toyota        26
Triumph        1
Volkswagen    22
Volvo          6
Name: company, dtype: int64

<IPython.core.display.Javascript object>

#### Get region from origin column
The region of origin is categorical. In the dataset it is ordinal (1 for USA, 2 for Euroope and 3 for Asia). It should be nominal data.

The function `get_region_names` will do this for us.

For later model training, we'll use one-hot-encoding to generate the new features.

In [9]:
from src.clean import get_region_names

get_region_names(df)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,name,company,region
0,18.0,8,307.0,130.0,3504.0,12.0,70,Chevrolet Chevelle Malibu,Chevrolet,USA
1,15.0,8,350.0,165.0,3693.0,11.5,70,Buick Skylark 320,Buick,USA
2,18.0,8,318.0,150.0,3436.0,11.0,70,Plymouth Satellite,Plymouth,USA
3,16.0,8,304.0,150.0,3433.0,12.0,70,Amc Rebel Sst,AMC,USA
4,17.0,8,302.0,140.0,3449.0,10.5,70,Ford Torino,Ford,USA


<IPython.core.display.Javascript object>

In [10]:
from src.utils import save_data

save_data(df, "interim", "data_cleaned.pkl")

<IPython.core.display.Javascript object>

Next, we move all the cleaning stpes to create a function `clean_dataset` in the module `clean.py`. 